<a href="https://colab.research.google.com/github/staghado/Exploration-Notebooks/blob/main/Zephyr-7b-hf-open-llm-Mistral-7B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Running Zephyr 7B Alpha on Google Colab

In [ ]:
# Install transformers from source - only needed for versions <= v4.34
%pip install git+https://github.com/huggingface/transformers.git
%pip install accelerate

## Load dependencies

In [2]:
import torch
from transformers import pipeline

## Download model and load pipeline

In [ ]:
pipe = pipeline("text-generation", model="HuggingFaceH4/zephyr-7b-alpha", torch_dtype=torch.bfloat16, device_map="auto")

## Prepare inputs

In [ ]:
# Each message can have 1 of 3 roles: "system" (to provide initial instructions), "user", or "assistant".
# For inference, make sure "user" is the role in the final message.
messages = [
    {
        "role": "system",
        "content": "You are a friendly chatbot who's objective is to be as helpful as possible. Always provide the answer as a JSON formatted output. Answer in English even if the input is given in French(including job titles).",
    },
    {"role": "user", "content": """
    Given the following email, extract the company name, position title, application status. Do not output anything else.

"""},
]
# We use the tokenizer's chat template to format each message - see https://huggingface.co/docs/transformers/main/en/chat_templating
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
print(prompt)

## Generate!

In [ ]:
outputs = pipe(prompt, max_new_tokens=100, do_sample=True, temperature=0.2, top_k=50, top_p=0.95)
print(outputs[0]["generated_text"])

In [ ]:
d = eval(outputs[0]["generated_text"].split('<|assistant|>')[-1])
d